### report of implementations

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
from classes2048 import state, tbl
from time import sleep
import tensorflow as tf
import numpy as np
from functions import mv
from pynput.keyboard import Key, Controller
import math
from IPython.display import clear_output
import os
keys= [Key.right,Key.down,Key.up,Key.left]

In [ ]:
game=state()
table=tbl(game)
keyboard = Controller()

In [ ]:
game.getprops()

In [ ]:
table.gettbl()

### Hyperparameters 

In [ ]:
# num_input_neurons = len(env.reset())
# num_ouptut_neurons = env.action_space.n

num_input_neurons = 16
num_ouptut_neurons = 4
num_hidden_layer_neurons_1 = 256

gamma = 0.99
max_epsilon = 0.9
epsilon = max_epsilon
min_epsilon = 0.01
epsilon_decay_factor = 0.001

num_episodes = 10

learning_rate = 0.001

### training the Q-network

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, kernel_initializer='random_uniform', activation='relu', input_dim = 16),
    tf.keras.layers.Dense(128, kernel_initializer='random_uniform',activation='relu'),

    tf.keras.layers.Dense(4, kernel_initializer='random_uniform',activation='relu')
#       tf.keras.layers.Dense(128, activation='relu', input_dim = 16),
#       tf.keras.layers.Dense(4, activation='relu')
    
])
opt=tf.keras.optimizers.Adam(lr=learning_rate)

model.compile(optimizer=opt,
              loss='mse',       # mean squared error
              metrics=['mae'])


In [ ]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

In [ ]:
reward_list = []
num_episodes=1
sleep(2)
for i in range(num_episodes):
    table.gettbl()
    tablestate = table.tablestate()
    reward_so_far = 0
    while game.statepredict()=='going':
        action, q_vals = np.array([np.argmax(model.predict(tablestate))]), model.predict(tablestate)
        
        if np.random.rand(1) < epsilon:
            action[0] = np.random.randint(4)
            
        mv(keys[action[0]] )
        sleep(0.3)
        table.gettbl()
        next_tablestate=table.tablestate()
        
        if np.array_equal(next_tablestate,tablestate):
            reward=-100
        else:
            reward = tablestate.sum()-next_tablestate.sum()
        
        if game.statepredict()=='lost':
            reward=-1000

        max_q=np.max(model.predict(tablestate))
        target_q_vals = q_vals
        target_q_vals[0, action[0]] = reward + gamma * max_q
        
        print(table.gettbl(),reward, keys[action[0]])
        model.fit(tablestate, target_q_vals)
        

        clear_output(wait=True)

        tablestate = next_tablestate
        reward_so_far += reward
        sleep(0.1)

    # epsilon decay
    game.statepredict(save=True, folder='./data/lost_games/')
    keyboard.press(Key.f5)
    sleep(0.02)
    keyboard.release(Key.f5)
    sleep(3)
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * math.exp(-epsilon_decay_factor * i)

    reward_list.append(reward_so_far)


In [ ]:
checkpoint_path = "checkpoints/"
checkpoint_dir = os.path.dirname(checkpoint_path)


# Create checkpoint callback
# cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)

model.save_weights('./checkpoints/')

#### in case we have wrong gamestate

In [ ]:
game.statepredict(save=True)
table.gettbl()

In [ ]:
game.fit()
game.save()

## testing the network

In [ ]:
num_episodes = 2
total_reward = []

# model = create_model()
# model.load_weights(latest)
# loss, acc = model.evaluate(test_images, test_labels)
 
sleep(2)
table.gettbl()

for _ in range(num_episodes):
    
    while game.statepredict()=='going':
        table.gettbl()
        tablestate = table.tablestate()
#         action = sess.run(best_action, feed_dict={input_obs: state})
        action= np.array([np.argmax(model.predict(tablestate))])
        mv(keys[action[0]])
        sleep(2)
        
        if np.array_equal(next_tablestate,tablestate):
            reward=-100
        else:
            reward = tablestate.sum()-next_tablestate.sum()
        
        if game.statepredict()=='lost':
            reward=-1000
            
       # state, reward, done, _ = env.step(action[0])
        total_reward.append(reward)

        sleep(0.1)


print("Average reward per episode: {}".format(total_reward/float(num_episodes)))